In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
               .appName('SparkByExamples.com') \
               .getOrCreate()

In [ ]:
data = [("James","","Smith",30,"M",60000),
        ("Michael","Rose","",50,"M",70000),
        ("Robert","","Williams",42,"",400000),
        ("Maria","Anne","Jones",38,"F",500000),
        ("Jen","Mary","Brown",45,"F",0)]

In [ ]:
columns = ["first_name","middle_name","last_name","Age","gender","salary"]
pysparkDF = spark.createDataFrame(data = data, schema = columns)
pysparkDF.printSchema()
pysparkDF.show(truncate=False)

In [ ]:
df = spark.read.csv("/mnt/10ac-batch-5/week9/reiten/unprocessed/original_set.csv", header=True, inferSchema=True)
df.show()

In [ ]:
import pandas as pd
f2 = pd.read_csv("/mnt/10ac-batch-5/week9/reiten/interim/clean_set.csv")
f2.head()

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Collinear Points")
sc = SparkContext('local',conf=conf)    
from pyspark.rdd import RDD
binary_wave_rdd = sc.binaryFiles('../data/wav/*.wav')

In [ ]:
def extract_sentences(df, id_len = 6, char_limit = 150):
        """
        Args:
            df: original dataframe
            col_name:
            id_len:
            char_limit:
            
        Returns: None
        """
        id = 1
        st_df = pd.DataFrame(columns = ['Id', 'Text'])
        texts = []
        ids = []
            
        for i in range(len(df)):
            articles = df.loc[i,"article"].split("።")
            for text in articles:
                if(len(text) < char_limit):
                    texts.append(text)
                    ids.append("0"*(id_len-(len(str(id))))+str(id))
                    id = id+1

        st_df['Id'] = ids
        st_df['Text'] = texts

        return st_df

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Collinear Points")
sc = SparkContext('local',conf=conf) 

In [ ]:
from pyspark.rdd import RDD
import librosa
import io
import numpy as np
import pandas as pd
from scipy.io.wavfile import write
import scipy.io.wavfile

In [ ]:
# Create a binary rdd file from the audio files
binary_wave_rdd = sc.binaryFiles('/mnt/10ac-batch-5/week9/reiten/unprocessed/*.wav')

In [ ]:
# Transfomer binary_wave_rdd to a tuple rdd with location of file and numpy array
rdd = binary_wave_rdd.map(lambda x : (x[0].split('/')[-1].split('.')[0], librosa.load(io.BytesIO(x[1]))))
rdd.collect()

In [ ]:
class CleanAudio():
    """Clean audio data by removing dead spaces, ...
    """

    def __init__(self):
        pass

    def normalize_audio(self, signal):
        feats_mean = np.mean(signal, axis=0)
        feats_std = np.std(signal, axis=0)
        signal = (signal - feats_mean) / (feats_std + 1e-14)
        return signal

In [ ]:
class Predict():
    def __init__(self):

        self.clean_audio = CleanAudio()

    def get_audio(self, audio_file):
        sr = 8000
        wav, rate = audio_file
        y = librosa.resample(wav, rate, sr)
        return y

    def get_clean_audio(self, wav):
        y = self.clean_audio.normalize_audio(wav)
        return y


def validate(rdd):
    
    predict = Predict()
    audio_file_rdd = rdd.map(lambda x : (x[0], predict.get_audio(x[1])))
    clean_audio_file_rdd = audio_file_rdd.map(lambda x : (x[0], predict.get_clean_audio(x[1])))
    return clean_audio_file_rdd
    

In [ ]:
clean_audio_file_rdd = validate(binary_wave_rdd)

# get collection of audio wave file and turn it to dictionary
coll_clean = clean_audio_file_rdd.collect()
dct_clean = dict((y, x) for y, x in coll_clean)

# overwrite clean audio to file

for i,j in dct_clean.items():
    scipy.io.wavfile.write('/mnt/10ac-batch-5/week9/reiten/unprocessed/'+i+'.wav', 8000,j)

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
               .appName('SparkByExamples.com') \
               .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/09 21:51:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df5 = spark.read.csv("/mnt/10ac-batch-5/week9/reiten/interim/clean_set.csv")
df5.show()

+----+------+--------------------+
| _c0|   _c1|                 _c2|
+----+------+--------------------+
|null|    Id|                Text|
|   0|000001|ብርሃን ፈይሳየኢትዮጵያ ቦክ...|
|   1|000002| የተሻለ ብቃት ያሳዩ ቦክሰ...|
|   2|000003|በቦክስ ስፖርት ከሚካሄዱት ...|
|   3|000004| የኢትዮጵያ ቦክስ ፌዴሬሽን...|
|   4|000005| በአጠቃላይ ክለቦቻቸውን ወ...|
|   5|000006| ውድድሩ የሚካሄደው በራስ ...|
|   6|000007| የቦክስ ስፖርት ከንክኪ ስ...|
|   7|000008| በመሆኑም በዚህ ረገድ ውድ...|
|   8|000009| ይኸውም ተወዳዳሪዎችና አሰ...|
|   9|000010| ከዚህ ባሻገር ውድድር የሚ...|
|  10|000011| ኮቪድ 19 ዓለም አቀፍ ወ...|
|  11|000012| ከእነዚህ ውድድሮች መካከል...|
|  12|000013| ውድድሮች ወደ እንቅስቃሴ ...|
|  13|000014| ኢትዮጵያም በዚህ ቻምፒዮና...|
|  14|000015| ቻምፒዮናው ጥር 4/2013...|
|  15|000016|                null|
|  16|000017| የአዲስ ዘመን ጋዜጣ ቀደም...|
|  17|000018|እኛም ከእነዚህ ዘገባዎች ጋ...|
|  18|000019| በዛሬው የአዲስ ዘመን ዱሮ...|
+----+------+--------------------+
only showing top 20 rows



In [5]:
rdd = df5.rdd.map(lambda loop: (
      loop["Id"],loop["Text"])
  )